# Fine-tune Llama 3 for Markdown Training

This notebook uses [Unsloth](https://github.com/unslothai/unsloth) to fine-tune a Llama 3 model on your custom Markdown dataset. Unsloth is optimized for free Colab GPUs (T4).

In [ ]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()
# Must install separately since Colab has torch 2.2.1, which breaks packages
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # Use new 8-bit quantization for Ampere GPUs (A100, H100)
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Use old quantization for Volta/Turing GPUs (T4, V100)
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

In [ ]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit", # Llama-3 8b 4bit quantized
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    use_gradient_checkpointing = "unsloth", # 4x longer context window + lower VRAM usage.
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

### Load Dataset
We will load your custom dataset directly here.

In [ ]:
from datasets import Dataset

# Your Custom Dataset injected directly
custom_data = [
    {
        "instruction": "How do I create a level 1 header in Markdown?",
        "input": "",
        "output": "# Heading Level 1"
    },
    {
        "instruction": "Create a level 2 header with the text 'Introduction' in Markdown.",
        "input": "",
        "output": "## Introduction"
    },
    {
        "instruction": "Show me an example of bold text in Markdown using asterisks.",
        "input": "",
        "output": "**Bold Text**"
    },
    {
        "instruction": "How do I italicize text using underscores in Markdown?",
        "input": "",
        "output": "_Italic Text_"
    },
    {
        "instruction": "Create an unordered list with three items: Apple, Banana, Orange in Markdown.",
        "input": "",
        "output": "- Apple\n- Banana\n- Orange"
    },
    {
        "instruction": "How do I create a link to Google in Markdown?",
        "input": "",
        "output": "[Google](https://www.google.com)"
    },
    {
        "instruction": "Insert an image with alt text 'Logo' and source 'logo.png' in Markdown.",
        "input": "",
        "output": "![Logo](logo.png)"
    },
    {
        "instruction": "Create a code block for Python code that prints 'Hello' in Markdown.",
        "input": "",
        "output": "```python\nprint('Hello')\n```"
    },
    {
        "instruction": "How do I make a blockquote in Markdown?",
        "input": "",
        "output": "> This is a blockquote."
    },
    {
        "instruction": "Create a table with two columns: Name and Age in Markdown.",
        "input": "",
        "output": "| Name | Age |\n| --- | --- |\n| Alice | 30 |\n| Bob | 25 |"
    },
    {
        "instruction": "How do I add a horizontal rule in Markdown?",
        "input": "",
        "output": "---"
    },
    {
        "instruction": "Show me how to do a strikethrough in Markdown.",
        "input": "",
        "output": "~~Strikethrough~~"
    },
    {
        "instruction": "Create a task list with one done item and one todo item in Markdown.",
        "input": "",
        "output": "- [x] Done item\n- [ ] Todo item"
    },
    {
        "instruction": "How do I escape a special character like an asterisk in Markdown?",
        "input": "",
        "output": "\\*"
    },
    {
        "instruction": "Create a link that has a title attribute in Markdown.",
        "input": "",
        "output": "[Link Text](https://example.com \"Title Text\")"
    },
    {
        "instruction": "How do I create a definition list in Markdown?",
        "input": "",
        "output": "Term\n: Definition"
    },
    {
        "instruction": "Show me how to footnote a sentence in Markdown.",
        "input": "",
        "output": "This is a sentence.[^1]\n\n[^1]: This is the footnote."
    },
    {
        "instruction": "How do I write a superscript in Markdown?",
        "input": "",
        "output": "X^2^"
    },
    {
        "instruction": "How do I write a subscript in Markdown?",
        "input": "",
        "output": "H~2~O"
    },
    {
        "instruction": "Create a collapse/expand details section in Markdown (using HTML).",
        "input": "",
        "output": "<details>\n<summary>Click to expand</summary>\n\nHidden content goes here.\n</details>"
    },
    {
        "instruction": "How do I highlight text (if supported) in Markdown?",
        "input": "",
        "output": "==Highlighted Text=="
    },
    {
        "instruction": "Create a nested list structure in Markdown.",
        "input": "",
        "output": "1. First Item\n   - Nested Item 1\n   - Nested Item 2\n2. Second Item"
    },
    {
        "instruction": "How can I comment out text in Markdown?",
        "input": "",
        "output": "<!-- This is a comment and will not be rendered -->"
    },
    {
        "instruction": "Create a mermaid class diagram example in Markdown.",
        "input": "",
        "output": "```mermaid\nclassDiagram\n    Animal <|-- Duck\n    Animal <|-- Fish\n    Animal <|-- Zebra\n    Animal : +int age\n    Animal : +String gender\n    class Duck{\n      +String beakColor\n      +swim()\n      +quack()\n    }\n    class Fish{\n      -int sizeInFeet\n      -canEat()\n    }\n```"
    },
    {
        "instruction": "What is the syntax for a math block (KaTeX/MathJax) in Markdown?",
        "input": "",
        "output": "$$ E = mc^2 $$"
    },
    {
        "instruction": "Explain Markdown syntax for Table of Contents.",
        "input": "",
        "output": "1. [Basic Syntax](#basic-syntax)\n2. [Extended Syntax](#extended-syntax)\n3. [Code and Syntax Highlighting](#code-and-syntax-highlighting)\n4. [Tables](#tables)\n5. [Advanced Features](#advanced-features)\n6. [Diagrams (Mermaid)](#diagrams-mermaid)\n7. [Document Templates](#document-templates)\n\n---"
    },
    {
        "instruction": "Provide examples of Table of Contents in Markdown.",
        "input": "",
        "output": "1. [Basic Syntax](#basic-syntax)\n2. [Extended Syntax](#extended-syntax)\n3. [Code and Syntax Highlighting](#code-and-syntax-highlighting)\n4. [Tables](#tables)\n5. [Advanced Features](#advanced-features)\n6. [Diagrams (Mermaid)](#diagrams-mermaid)\n7. [Document Templates](#document-templates)\n\n---"
    },
    {
        "instruction": "Explain Markdown syntax for Basic Syntax.",
        "input": "",
        "output": "### Headers\n\nHeaders are created using the `#` symbol. The number of `#` symbols corresponds to the header level (1-6).\n\n```markdown\n# Heading Level 1"
    },
    {
        "instruction": "Provide examples of Basic Syntax in Markdown.",
        "input": "",
        "output": "### Headers\n\nHeaders are created using the `#` symbol. The number of `#` symbols corresponds to the header level (1-6).\n\n```markdown\n# Heading Level 1"
    },
    {
        "instruction": "Explain Markdown syntax for Heading Level 2.",
        "input": "",
        "output": "### Heading Level 3\n#### Heading Level 4\n##### Heading Level 5\n###### Heading Level 6\n```\n\n**Alternative Syntax (H1 and H2 only):**\n\n```markdown\nHeading Level 1\n===============\n\nHeading Level 2\n---------------\n```\n\n### Paragraphs and Line Breaks\n\nParagraphs are separated by a blank line. To create a line break within a paragraph (soft break), refer to the end of the line with two or more spaces or a backslash `\\`.\n\n```markdown\nThis is a paragraph.\nIt continues on the same line if there is no blank line between.\n\nThis is a new paragraph.\n\nThis line ends with two spaces.  \nThis is a new line in the same paragraph.\n```\n\n### Emphasis\n\n- **Bold**: Use `**text**` or `__text__`.\n- *Italic*: Use `*text*` or `_text_`.\n- ***Bold and Italic***: Use `***text***` or `___text___`.\n\n```markdown\n**Bold Text** using asterisks.\n__Bold Text__ using underscores.\n\n*Italic Text* using asterisks.\n_Italic Text_ using underscores.\n\n***Bold and Italic*** text.\n```\n\n### Blockquotes\n\nBlockquotes are indicated by `>`. They can be nested.\n\n```markdown\n> This is a blockquote.\n>\n> > This is a nested blockquote.\n>\n> Back to the first level.\n```\n\n### Lists\n\n#### Unordered Lists\nUse `*`, `-`, or `+`.\n\n```markdown\n- Item 1\n- Item 2\n  - Sub-item 2a\n  - Sub-item 2b\n* Item 3\n+ Item 4\n```\n\n#### Ordered Lists\nUse numbers followed by a period.\n\n```markdown\n1. First item\n2. Second item\n3. Third item\n   1. Indented item\n   2. Indented item\n```\n\n---"
    },
    {
        "instruction": "Provide examples of Heading Level 2 in Markdown.",
        "input": "",
        "output": "### Heading Level 3\n#### Heading Level 4\n##### Heading Level 5\n###### Heading Level 6\n```\n\n**Alternative Syntax (H1 and H2 only):**\n\n```markdown\nHeading Level 1\n===============\n\nHeading Level 2\n---------------\n```\n\n### Paragraphs and Line Breaks\n\nParagraphs are separated by a blank line. To create a line break within a paragraph (soft break), refer to the end of the line with two or more spaces or a backslash `\\`.\n\n```markdown\nThis is a paragraph.\nIt continues on the same line if there is no blank line between.\n\nThis is a new paragraph.\n\nThis line ends with two spaces.  \nThis is a new line in the same paragraph.\n```\n\n### Emphasis\n\n- **Bold**: Use `**text**` or `__text__`.\n- *Italic*: Use `*text*` or `_text_`.\n- ***Bold and Italic***: Use `***text***` or `___text___`.\n\n```markdown\n**Bold Text** using asterisks.\n__Bold Text__ using underscores.\n\n*Italic Text* using asterisks.\n_Italic Text_ using underscores.\n\n***Bold and Italic*** text.\n```\n\n### Blockquotes\n\nBlockquotes are indicated by `>`. They can be nested.\n\n```markdown\n> This is a blockquote.\n>\n> > This is a nested blockquote.\n>\n> Back to the first level.\n```\n\n### Lists\n\n#### Unordered Lists\nUse `*`, `-`, or `+`.\n\n```markdown\n- Item 1\n- Item 2\n  - Sub-item 2a\n  - Sub-item 2b\n* Item 3\n+ Item 4\n```\n\n#### Ordered Lists\nUse numbers followed by a period.\n\n```markdown\n1. First item\n2. Second item\n3. Third item\n   1. Indented item\n   2. Indented item\n```\n\n---"
    },
    {
        "instruction": "Explain Markdown syntax for Extended Syntax.",
        "input": "",
        "output": "### Strikethrough\n\nUse `~~` to cross out text.\n\n```markdown\n~~This text is struck through.~~\n```\n\n### Task Lists\n\nTask lists allow you to create checklists.\n\n```markdown\n- [x] Completed task\n- [ ] Incomplete task\n- [ ] Another incomplete task\n```\n\n### Links\n\n#### Inline Links\n\n```markdown\n[Link Text](https://www.example.com \"Optional Title\")\n```\n\n#### Reference Links\n\nKeep the source readable by defining links elsewhere.\n\n```markdown\nThis is a link to [Google][1].\n\n[1]: https://www.google.com\n```\n\n#### Auto-links\n\n```markdown\n<https://www.example.com>\n<fake@example.com>\n```\n\n### Images\n\nSimilar to links, but with a preceding `!`.\n\n```markdown\n![Alt Text](https://via.placeholder.com/150 \"Image Title\")\n```\n\nWith link:\n\n```markdown\n[![Alt Text](https://via.placeholder.com/150)](https://www.example.com)\n```\n\n---"
    },
    {
        "instruction": "Provide examples of Extended Syntax in Markdown.",
        "input": "",
        "output": "### Strikethrough\n\nUse `~~` to cross out text.\n\n```markdown\n~~This text is struck through.~~\n```\n\n### Task Lists\n\nTask lists allow you to create checklists.\n\n```markdown\n- [x] Completed task\n- [ ] Incomplete task\n- [ ] Another incomplete task\n```\n\n### Links\n\n#### Inline Links\n\n```markdown\n[Link Text](https://www.example.com \"Optional Title\")\n```\n\n#### Reference Links\n\nKeep the source readable by defining links elsewhere.\n\n```markdown\nThis is a link to [Google][1].\n\n[1]: https://www.google.com\n```\n\n#### Auto-links\n\n```markdown\n<https://www.example.com>\n<fake@example.com>\n```\n\n### Images\n\nSimilar to links, but with a preceding `!`.\n\n```markdown\n![Alt Text](https://via.placeholder.com/150 \"Image Title\")\n```\n\nWith link:\n\n```markdown\n[![Alt Text](https://via.placeholder.com/150)](https://www.example.com)\n```\n\n---"
    },
    {
        "instruction": "Explain Markdown syntax for Code and Syntax Highlighting.",
        "input": "",
        "output": "### Inline Code\n\nWrap code in backticks `` ` ``.\n\n```markdown\nUse the `printf()` function to print text.\n```\n\n### Fenced Code Blocks\n\nUse three backticks ``` or tildes `~~~` to create a code block. Specify the language for syntax highlighting.\n\n#### Python Example\n\n```python\ndef hello_world():\n    print(\"Hello, World!\")\n\nif __name__ == \"__main__\":\n    hello_world()\n```\n\n#### JavaScript Example\n\n```javascript\nconst greet = (name) => {\n    console.log(`Hello, ${name}!`);\n};\n\ngreet('Developer');\n```\n\n#### Go Example\n\n```go\npackage main\n\nimport \"fmt\"\n\nfunc main() {\n    fmt.Println(\"Hello, World!\")\n}\n```\n\n#### JSON Example\n\n```json\n{\n  \"key\": \"value\",\n  \"array\": [1, 2, 3],\n  \"nested\": {\n    \"boolean\": true\n  }\n}\n```\n\n#### Bash/Shell Example\n\n```bash\n#!/bin/bash\necho \"Installing dependencies...\"\nnpm install\n```\n\n---"
    },
    {
        "instruction": "Provide examples of Code and Syntax Highlighting in Markdown.",
        "input": "",
        "output": "### Inline Code\n\nWrap code in backticks `` ` ``.\n\n```markdown\nUse the `printf()` function to print text.\n```\n\n### Fenced Code Blocks\n\nUse three backticks ``` or tildes `~~~` to create a code block. Specify the language for syntax highlighting.\n\n#### Python Example\n\n```python\ndef hello_world():\n    print(\"Hello, World!\")\n\nif __name__ == \"__main__\":\n    hello_world()\n```\n\n#### JavaScript Example\n\n```javascript\nconst greet = (name) => {\n    console.log(`Hello, ${name}!`);\n};\n\ngreet('Developer');\n```\n\n#### Go Example\n\n```go\npackage main\n\nimport \"fmt\"\n\nfunc main() {\n    fmt.Println(\"Hello, World!\")\n}\n```\n\n#### JSON Example\n\n```json\n{\n  \"key\": \"value\",\n  \"array\": [1, 2, 3],\n  \"nested\": {\n    \"boolean\": true\n  }\n}\n```\n\n#### Bash/Shell Example\n\n```bash\n#!/bin/bash\necho \"Installing dependencies...\"\nnpm install\n```\n\n---"
    },
    {
        "instruction": "Explain Markdown syntax for Tables.",
        "input": "",
        "output": "Tables are created using `|` and `-`. Colons `:` control alignment.\n\n```markdown\n| Left Align | Center Align | Right Align |\n| :---       | :----:       | ---:        |\n| Row 1 Col 1| Row 1 Col 2  | Row 1 Col 3 |\n| Row 2 Col 1| Row 2 Col 2  | Row 2 Col 3 |\n| Text       | Text         | Text        |\n```\n\n**Rendered Output:**\n\n| Left Align | Center Align | Right Align |\n| :---       | :----:       | ---:        |\n| Row 1 Col 1| Row 1 Col 2  | Row 1 Col 3 |\n| Row 2 Col 1| Row 2 Col 2  | Row 2 Col 3 |\n| Text       | Text         | Text        |\n\n---"
    },
    {
        "instruction": "Provide examples of Tables in Markdown.",
        "input": "",
        "output": "Tables are created using `|` and `-`. Colons `:` control alignment.\n\n```markdown\n| Left Align | Center Align | Right Align |\n| :---       | :----:       | ---:        |\n| Row 1 Col 1| Row 1 Col 2  | Row 1 Col 3 |\n| Row 2 Col 1| Row 2 Col 2  | Row 2 Col 3 |\n| Text       | Text         | Text        |\n```\n\n**Rendered Output:**\n\n| Left Align | Center Align | Right Align |\n| :---       | :----:       | ---:        |\n| Row 1 Col 1| Row 1 Col 2  | Row 1 Col 3 |\n| Row 2 Col 1| Row 2 Col 2  | Row 2 Col 3 |\n| Text       | Text         | Text        |\n\n---"
    },
    {
        "instruction": "Explain Markdown syntax for Advanced Features.",
        "input": "",
        "output": "### Horizontal Rules\n\nThree or more asterisks, dashes, or underscores.\n\n```markdown\n---\n***\n___\n```\n\n### Escaping Characters\n\nUse a backslash `\\` to escape special Markdown characters.\n\n```markdown\n\\* This is not a list item.\n\\# This is not a header.\n\\[This is not a link\\]\n```\n\n### Footnotes\n\n```markdown\nHere is a sentence with a footnote.[^1]\n\n[^1]: This is the footnote text.\n```\n\n### Definition Lists\n\n```markdown\nTerm 1\n: Definition 1\n\nTerm 2\n: Definition 2\n```\n\n### Subscript and Superscript\n\n```markdown\nH~2~O  (Subscript)\nX^2^   (Superscript)\n```\n\n### HTML in Markdown\n\nYou can use raw HTML for more control (if supported by the renderer).\n\n```html\n<dl>\n  <dt>Definition List</dt>\n  <dd>Is something people use sometimes.</dd>\n  <dt>Markdown in HTML</dt>\n  <dd>Does *not* work **very** well. Use HTML <em>tags</em>.</dd>\n</dl>\n```\n\n---"
    },
    {
        "instruction": "Provide examples of Advanced Features in Markdown.",
        "input": "",
        "output": "### Horizontal Rules\n\nThree or more asterisks, dashes, or underscores.\n\n```markdown\n---\n***\n___\n```\n\n### Escaping Characters\n\nUse a backslash `\\` to escape special Markdown characters.\n\n```markdown\n\\* This is not a list item.\n\\# This is not a header.\n\\[This is not a link\\]\n```\n\n### Footnotes\n\n```markdown\nHere is a sentence with a footnote.[^1]\n\n[^1]: This is the footnote text.\n```\n\n### Definition Lists\n\n```markdown\nTerm 1\n: Definition 1\n\nTerm 2\n: Definition 2\n```\n\n### Subscript and Superscript\n\n```markdown\nH~2~O  (Subscript)\nX^2^   (Superscript)\n```\n\n### HTML in Markdown\n\nYou can use raw HTML for more control (if supported by the renderer).\n\n```html\n<dl>\n  <dt>Definition List</dt>\n  <dd>Is something people use sometimes.</dd>\n  <dt>Markdown in HTML</dt>\n  <dd>Does *not* work **very** well. Use HTML <em>tags</em>.</dd>\n</dl>\n```\n\n---"
    },
    {
        "instruction": "Explain Markdown syntax for Diagrams (Mermaid).",
        "input": "",
        "output": "Many modern Markdown editors support Mermaid diagrams.\n\n### Flowchart\n\n```mermaid\ngraph TD;\n    A-->B;\n    A-->C;\n    B-->D;\n    C-->D;\n```\n\n### Sequence Diagram\n\n```mermaid\nsequenceDiagram\n    participant Alice\n    participant Bob\n    Alice->>Bob: Hello Bob, how are you?\n    Bob-->>Alice: I am good thanks!\n```\n\n### Gantt Chart\n\n```mermaid\ngantt\n    title A Gantt Diagram\n    dateFormat  YYYY-MM-DD\n    section Section\n    A task           :a1, 2014-01-01, 30d\n    Another task     :after a1  , 20d\n```\n\n---"
    },
    {
        "instruction": "Provide examples of Diagrams (Mermaid) in Markdown.",
        "input": "",
        "output": "Many modern Markdown editors support Mermaid diagrams.\n\n### Flowchart\n\n```mermaid\ngraph TD;\n    A-->B;\n    A-->C;\n    B-->D;\n    C-->D;\n```\n\n### Sequence Diagram\n\n```mermaid\nsequenceDiagram\n    participant Alice\n    participant Bob\n    Alice->>Bob: Hello Bob, how are you?\n    Bob-->>Alice: I am good thanks!\n```\n\n### Gantt Chart\n\n```mermaid\ngantt\n    title A Gantt Diagram\n    dateFormat  YYYY-MM-DD\n    section Section\n    A task           :a1, 2014-01-01, 30d\n    Another task     :after a1  , 20d\n```\n\n---"
    },
    {
        "instruction": "Explain Markdown syntax for Document Templates.",
        "input": "",
        "output": "### Example 1: Project README\n\n```markdown\n# Project Name\n\n> Short description of the project.\n\n[![Build Status](https://img.shields.io/badge/build-passing-brightgreen)](https://example.com)\n[![License](https://img.shields.io/badge/license-MIT-blue)](LICENSE)"
    },
    {
        "instruction": "Provide examples of Document Templates in Markdown.",
        "input": "",
        "output": "### Example 1: Project README\n\n```markdown\n# Project Name\n\n> Short description of the project.\n\n[![Build Status](https://img.shields.io/badge/build-passing-brightgreen)](https://example.com)\n[![License](https://img.shields.io/badge/license-MIT-blue)](LICENSE)"
    },
    {
        "instruction": "Explain Markdown syntax for Features.",
        "input": "",
        "output": "- Feature 1: Description\n- Feature 2: Description\n- Feature 3: Description"
    },
    {
        "instruction": "Provide examples of Features in Markdown.",
        "input": "",
        "output": "- Feature 1: Description\n- Feature 2: Description\n- Feature 3: Description"
    },
    {
        "instruction": "Explain Markdown syntax for Installation.",
        "input": "",
        "output": "```bash\nnpm install project-name\n```"
    },
    {
        "instruction": "Provide examples of Installation in Markdown.",
        "input": "",
        "output": "```bash\nnpm install project-name\n```"
    },
    {
        "instruction": "Explain Markdown syntax for Usage.",
        "input": "",
        "output": "```javascript\nimport { functionName } from 'project-name';\n\nfunctionName();\n```"
    },
    {
        "instruction": "Provide examples of Usage in Markdown.",
        "input": "",
        "output": "```javascript\nimport { functionName } from 'project-name';\n\nfunctionName();\n```"
    },
    {
        "instruction": "Explain Markdown syntax for Contributing.",
        "input": "",
        "output": "1. Fork the repository\n2. Create a feature branch\n3. Commit your changes\n4. Push to the branch\n5. Open a Pull Request"
    },
    {
        "instruction": "Provide examples of Contributing in Markdown.",
        "input": "",
        "output": "1. Fork the repository\n2. Create a feature branch\n3. Commit your changes\n4. Push to the branch\n5. Open a Pull Request"
    },
    {
        "instruction": "Explain Markdown syntax for License.",
        "input": "",
        "output": "MIT\n```\n\n### Example 2: API Documentation\n\n```markdown\n# API Reference"
    },
    {
        "instruction": "Provide examples of License in Markdown.",
        "input": "",
        "output": "MIT\n```\n\n### Example 2: API Documentation\n\n```markdown\n# API Reference"
    },
    {
        "instruction": "Explain Markdown syntax for Authentication.",
        "input": "",
        "output": "All API requests require an API key.\n\n`Authorization: Bearer <YOUR_API_KEY>`"
    },
    {
        "instruction": "Provide examples of Authentication in Markdown.",
        "input": "",
        "output": "All API requests require an API key.\n\n`Authorization: Bearer <YOUR_API_KEY>`"
    },
    {
        "instruction": "Explain Markdown syntax for Endpoints.",
        "input": "",
        "output": "### Get User\n\nRetrieves the current user's profile.\n\n**Request:**\n\n`GET /api/v1/user`\n\n**Response:**\n\n```json\n{\n  \"id\": 123,\n  \"name\": \"Jane Doe\",\n  \"email\": \"jane@example.com\"\n}\n```\n\n### Create Post\n\nCreates a new blog post.\n\n**Request:**\n\n`POST /api/v1/posts`\n\n| Parameter | Type   | Required | Description |\n| :---      | :---   | :---:    | :---        |\n| title     | string | Yes      | Post title  |\n| body      | string | Yes      | Post content|\n\n**Response:**\n\n`201 Created`\n```\n\n### Example 3: Blog Post\n\n```markdown\n# The Future of Web Development\n\n*Published on October 27, 2023 by Author*\n\nWeb development is constantly evolving. In this post, we will explore..."
    },
    {
        "instruction": "Provide examples of Endpoints in Markdown.",
        "input": "",
        "output": "### Get User\n\nRetrieves the current user's profile.\n\n**Request:**\n\n`GET /api/v1/user`\n\n**Response:**\n\n```json\n{\n  \"id\": 123,\n  \"name\": \"Jane Doe\",\n  \"email\": \"jane@example.com\"\n}\n```\n\n### Create Post\n\nCreates a new blog post.\n\n**Request:**\n\n`POST /api/v1/posts`\n\n| Parameter | Type   | Required | Description |\n| :---      | :---   | :---:    | :---        |\n| title     | string | Yes      | Post title  |\n| body      | string | Yes      | Post content|\n\n**Response:**\n\n`201 Created`\n```\n\n### Example 3: Blog Post\n\n```markdown\n# The Future of Web Development\n\n*Published on October 27, 2023 by Author*\n\nWeb development is constantly evolving. In this post, we will explore..."
    },
    {
        "instruction": "Explain Markdown syntax for The Rise of AI.",
        "input": "",
        "output": "Artificial Intelligence is changing how we write code.\n\n> \"AI will not replace developers, but developers using AI will replace those who don't.\""
    },
    {
        "instruction": "Provide examples of The Rise of AI in Markdown.",
        "input": "",
        "output": "Artificial Intelligence is changing how we write code.\n\n> \"AI will not replace developers, but developers using AI will replace those who don't.\""
    },
    {
        "instruction": "Explain Markdown syntax for Conclusion.",
        "input": "",
        "output": "The future is bright.\n\n---\n\n*Tags: #webdev, #ai, #future*\n```\n\n---\n\n**End of Guide**"
    },
    {
        "instruction": "Provide examples of Conclusion in Markdown.",
        "input": "",
        "output": "The future is bright.\n\n---\n\n*Tags: #webdev, #ai, #future*\n```\n\n---\n\n**End of Guide**"
    },
    {
        "instruction": "Generate a comprehensive Markdown cheat sheet.",
        "input": "",
        "output": "# Comprehensive Markdown Guide: Syntax, Examples, and Best Practices\n\nThis document serves as a comprehensive reference for Markdown syntax, including standard CommonMark features and GitHub Flavored Markdown (GFM) extensions. It is designed to be a training resource for understanding Markdown structure, formatting, and usage patterns.\n\n## Table of Contents\n\n1. [Basic Syntax](#basic-syntax)\n2. [Extended Syntax](#extended-syntax)\n3. [Code and Syntax Highlighting](#code-and-syntax-highlighting)\n4. [Tables](#tables)\n5. [Advanced Features](#advanced-features)\n6. [Diagrams (Mermaid)](#diagrams-mermaid)\n7. [Document Templates](#document-templates)\n\n---\n\n## 1. Basic Syntax\n\n### Headers\n\nHeaders are created using the `#` symbol. The number of `#` symbols corresponds to the header level (1-6).\n\n```markdown\n# Heading Level 1\n## Heading Level 2\n### Heading Level 3\n#### Heading Level 4\n##### Heading Level 5\n###### Heading Level 6\n```\n\n**Alternative Syntax (H1 and H2 only):**\n\n```markdown\nHeading Level 1\n===============\n\nHeading Level 2\n---------------\n```\n\n### Paragraphs and Line Breaks\n\nParagraphs are separated by a blank line. To create a line break within a paragraph (soft break), refer to the end of the line with two or more spaces or a backslash `\\`.\n\n```markdown\nThis is a paragraph.\nIt continues on the same line if there is no blank line between.\n\nThis is a new paragraph.\n\nThis line ends with two spaces.  \nThis is a new line in the same paragraph.\n```\n\n### Emphasis\n\n- **Bold**: Use `**text**` or `__text__`.\n- *Italic*: Use `*text*` or `_text_`.\n- ***Bold and Italic***: Use `***text***` or `___text___`.\n\n```markdown\n**Bold Text** using asterisks.\n__Bold Text__ using underscores.\n\n*Italic Text* using asterisks.\n_Italic Text_ using underscores.\n\n***Bold and Italic*** text.\n```\n\n### Blockquotes\n\nBlockquotes are indicated by `>`. They can be nested.\n\n```markdown\n> This is a blockquote.\n>\n> > This is a nested blockquote.\n>\n> Back to the first level.\n```\n\n### Lists\n\n#### Unordered Lists\nUse `*`, `-`, or `+`.\n\n```markdown\n- Item 1\n- Item 2\n  - Sub-item 2a\n  - Sub-item 2b\n* Item 3\n+ Item 4\n```\n\n#### Ordered Lists\nUse numbers followed by a period.\n\n```markdown\n1. First item\n2. Second item\n3. Third item\n   1. Indented item\n   2. Indented item\n```\n\n---\n\n## 2. Extended Syntax\n\n### Strikethrough\n\nUse `~~` to cross out text.\n\n```markdown\n~~This text is struck through.~~\n```\n\n### Task Lists\n\nTask lists allow you to create checklists.\n\n```markdown\n- [x] Completed task\n- [ ] Incomplete task\n- [ ] Another incomplete task\n```\n\n### Links\n\n#### Inline Links\n\n```markdown\n[Link Text](https://www.example.com \"Optional Title\")\n```\n\n#### Reference Links\n\nKeep the source readable by defining links elsewhere.\n\n```markdown\nThis is a link to [Google][1].\n\n[1]: https://www.google.com\n```\n\n#### Auto-links\n\n```markdown\n<https://www.example.com>\n<fake@example.com>\n```\n\n### Images\n\nSimilar to links, but with a preceding `!`.\n\n```markdown\n![Alt Text](https://via.placeholder.com/150 \"Image Title\")\n```\n\nWith link:\n\n```markdown\n[![Alt Text](https://via.placeholder.com/150)](https://www.example.com)\n```\n\n---\n\n## 3. Code and Syntax Highlighting\n\n### Inline Code\n\nWrap code in backticks `` ` ``.\n\n```markdown\nUse the `printf()` function to print text.\n```\n\n### Fenced Code Blocks\n\nUse three backticks ``` or tildes `~~~` to create a code block. Specify the language for syntax highlighting.\n\n#### Python Example\n\n```python\ndef hello_world():\n    print(\"Hello, World!\")\n\nif __name__ == \"__main__\":\n    hello_world()\n```\n\n#### JavaScript Example\n\n```javascript\nconst greet = (name) => {\n    console.log(`Hello, ${name}!`);\n};\n\ngreet('Developer');\n```\n\n#### Go Example\n\n```go\npackage main\n\nimport \"fmt\"\n\nfunc main() {\n    fmt.Println(\"Hello, World!\")\n}\n```\n\n#### JSON Example\n\n```json\n{\n  \"key\": \"value\",\n  \"array\": [1, 2, 3],\n  \"nested\": {\n    \"boolean\": true\n  }\n}\n```\n\n#### Bash/Shell Example\n\n```bash\n#!/bin/bash\necho \"Installing dependencies...\"\nnpm install\n```\n\n---\n\n## 4. Tables\n\nTables are created using `|` and `-`. Colons `:` control alignment.\n\n```markdown\n| Left Align | Center Align | Right Align |\n| :---       | :----:       | ---:        |\n| Row 1 Col 1| Row 1 Col 2  | Row 1 Col 3 |\n| Row 2 Col 1| Row 2 Col 2  | Row 2 Col 3 |\n| Text       | Text         | Text        |\n```\n\n**Rendered Output:**\n\n| Left Align | Center Align | Right Align |\n| :---       | :----:       | ---:        |\n| Row 1 Col 1| Row 1 Col 2  | Row 1 Col 3 |\n| Row 2 Col 1| Row 2 Col 2  | Row 2 Col 3 |\n| Text       | Text         | Text        |\n\n---\n\n## 5. Advanced Features\n\n### Horizontal Rules\n\nThree or more asterisks, dashes, or underscores.\n\n```markdown\n---\n***\n___\n```\n\n### Escaping Characters\n\nUse a backslash `\\` to escape special Markdown characters.\n\n```markdown\n\\* This is not a list item.\n\\# This is not a header.\n\\[This is not a link\\]\n```\n\n### Footnotes\n\n```markdown\nHere is a sentence with a footnote.[^1]\n\n[^1]: This is the footnote text.\n```\n\n### Definition Lists\n\n```markdown\nTerm 1\n: Definition 1\n\nTerm 2\n: Definition 2\n```\n\n### Subscript and Superscript\n\n```markdown\nH~2~O  (Subscript)\nX^2^   (Superscript)\n```\n\n### HTML in Markdown\n\nYou can use raw HTML for more control (if supported by the renderer).\n\n```html\n<dl>\n  <dt>Definition List</dt>\n  <dd>Is something people use sometimes.</dd>\n  <dt>Markdown in HTML</dt>\n  <dd>Does *not* work **very** well. Use HTML <em>tags</em>.</dd>\n</dl>\n```\n\n---\n\n## 6. Diagrams (Mermaid)\n\nMany modern Markdown editors support Mermaid diagrams.\n\n### Flowchart\n\n```mermaid\ngraph TD;\n    A-->B;\n    A-->C;\n    B-->D;\n    C-->D;\n```\n\n### Sequence Diagram\n\n```mermaid\nsequenceDiagram\n    participant Alice\n    participant Bob\n    Alice->>Bob: Hello Bob, how are you?\n    Bob-->>Alice: I am good thanks!\n```\n\n### Gantt Chart\n\n```mermaid\ngantt\n    title A Gantt Diagram\n    dateFormat  YYYY-MM-DD\n    section Section\n    A task           :a1, 2014-01-01, 30d\n    Another task     :after a1  , 20d\n```\n\n---\n\n## 7. Document Templates\n\n### Example 1: Project README\n\n```markdown\n# Project Name\n\n> Short description of the project.\n\n[![Build Status](https://img.shields.io/badge/build-passing-brightgreen)](https://example.com)\n[![License](https://img.shields.io/badge/license-MIT-blue)](LICENSE)\n\n## Features\n\n- Feature 1: Description\n- Feature 2: Description\n- Feature 3: Description\n\n## Installation\n\n```bash\nnpm install project-name\n```\n\n## Usage\n\n```javascript\nimport { functionName } from 'project-name';\n\nfunctionName();\n```\n\n## Contributing\n\n1. Fork the repository\n2. Create a feature branch\n3. Commit your changes\n4. Push to the branch\n5. Open a Pull Request\n\n## License\n\nMIT\n```\n\n### Example 2: API Documentation\n\n```markdown\n# API Reference\n\n## Authentication\n\nAll API requests require an API key.\n\n`Authorization: Bearer <YOUR_API_KEY>`\n\n## Endpoints\n\n### Get User\n\nRetrieves the current user's profile.\n\n**Request:**\n\n`GET /api/v1/user`\n\n**Response:**\n\n```json\n{\n  \"id\": 123,\n  \"name\": \"Jane Doe\",\n  \"email\": \"jane@example.com\"\n}\n```\n\n### Create Post\n\nCreates a new blog post.\n\n**Request:**\n\n`POST /api/v1/posts`\n\n| Parameter | Type   | Required | Description |\n| :---      | :---   | :---:    | :---        |\n| title     | string | Yes      | Post title  |\n| body      | string | Yes      | Post content|\n\n**Response:**\n\n`201 Created`\n```\n\n### Example 3: Blog Post\n\n```markdown\n# The Future of Web Development\n\n*Published on October 27, 2023 by Author*\n\nWeb development is constantly evolving. In this post, we will explore...\n\n## The Rise of AI\n\nArtificial Intelligence is changing how we write code.\n\n> \"AI will not replace developers, but developers using AI will replace those who don't.\"\n\n## Conclusion\n\nThe future is bright.\n\n---\n\n*Tags: #webdev, #ai, #future*\n```\n\n---\n\n**End of Guide**\n"
    }
]

# Convert to HuggingFace Dataset format
dataset = Dataset.from_list(custom_data)

alpaca_prompt = """Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

EOS_TOKEN = tokenizer.eos_token # Must add EOS_TOKEN
def formatting_prompts_func(examples):
    instructions = examples["instruction"]
    inputs       = examples["input"]
    outputs      = examples["output"]
    texts = []
    for instruction, input, output in zip(instructions, inputs, outputs):
        # Must add EOS_TOKEN, otherwise your generation will go on forever!
        text = alpaca_prompt.format(instruction, input, output) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts, }
pass

# Apply formatting
dataset = dataset.map(formatting_prompts_func, batched = True)

### Train the Model

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, # Can make training 5x faster for short sequences.
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        max_steps = 60, # Adjusted for small dataset (1 epoch approx)
        learning_rate = 2e-4,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
)

In [ ]:
trainer_stats = trainer.train()

### Inference (Test It)

In [ ]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    alpaca_prompt.format(
        "How do I make a table in Markdown?", # Instruction
        "", # Input
        "", # Output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
tokenizer.batch_decode(outputs)

### Save to GGUF
This will save the model in a format you can use with Ollama.

In [ ]:
# Save to 8bit Q8_0
model.save_pretrained_gguf("model_gguf", tokenizer, quantization_method = "q8_0")
# model.save_pretrained_gguf("model_gguf", tokenizer, quantization_method = "f16")